<a href="https://colab.research.google.com/github/sujayrittikar/NLP_Basics/blob/main/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spam or Ham

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('smsspamcollection.tsv', sep='\t')

In [12]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [4]:
from sklearn.model_selection import train_test_split

In [13]:
# X feature data
X = df['message']
# y is label
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
count_vect = CountVectorizer()

In [15]:
# Fit the Vectorizer to Data (builds vocab, counts no. of words...)
# Transform the original text message -> Forms numerical vector
X_train_counts = count_vect.fit_transform(X_train)

In [16]:
X_train_counts

<3900x7155 sparse matrix of type '<class 'numpy.int64'>'
	with 51338 stored elements in Compressed Sparse Row format>

In [11]:
X_train.shape

(3900, 1)

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

In [21]:
tfidf_transformer = TfidfTransformer()

In [23]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [24]:
X_train_tfidf.shape

(3900, 7155)

# Direct Vectorization
-> Count Vectorizer + Tfidf transformer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vectorizer = TfidfVectorizer()

In [27]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [28]:
from sklearn.svm import LinearSVC

In [29]:
clf = LinearSVC()

In [30]:
clf.fit(X_train_tfidf, y_train)

LinearSVC()

# Pipelining Everything

In [31]:
from sklearn.pipeline import Pipeline

In [32]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [33]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [34]:
predictions = text_clf.predict(X_test)

In [35]:
from sklearn.metrics import confusion_matrix, classification_report

In [36]:
print(confusion_matrix(y_test, predictions))

[[1437    5]
 [  20  210]]


In [37]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1442
        spam       0.98      0.91      0.94       230

    accuracy                           0.99      1672
   macro avg       0.98      0.95      0.97      1672
weighted avg       0.98      0.99      0.98      1672



In [39]:
from sklearn import metrics
metrics.accuracy_score(y_test, predictions)

0.9850478468899522

In [40]:
text_clf.predict(["Hey, how are you?"])

array(['ham'], dtype=object)

In [41]:
text_clf.predict(["Congrats you've been selected to earn $1 million from Ricord Inc. Text your phone number and SIN to 2355."])

array(['spam'], dtype=object)